In [1]:
import os, re, copy
from pathlib import Path
import pandas as pd
import requests 
from stop_words import get_stop_words
import warnings 
from pathlib import Path
from bs4 import BeautifulSoup 
import json
import os
import re
from pprint import pprint
import argparse
import ast
from pathlib import Path
import pathlib
import re
import pprint 
import argparse
p = Path('.')
warnings.filterwarnings('ignore')
pd.set_option('display.max_colwidth', None)

In [2]:
json_string='''@article{mohanram2005separating,
  title={Separating winners from losers among lowbook-to-market stocks using financial statement analysis},
  author={Mohanram, Partha S},
  journal={Review of accounting studies},
  volume={10},
  pages={133--170},
  year={2005},
  publisher={Springer}
}'''

url= f'https://scholar.google.com/scholar?cites=11874547788344227968&as_sdt=5,47&sciodt=0,47&hl=en' # user manual input

collect_dict_values= {}
for item in json_string.split('\n'):
    ii= item.split('=')
    if 'year' in [i.strip() for i in ii ]:
        collect_dict_values['year']=re.sub('\W+','-', item.split('=')[1])
    if 'title' in [i.strip() for i in ii ]:
        collect_dict_values['title']=re.sub('\W+','-', item.split('=')[1])
    if 'author' in [i.strip() for i in ii ]:
        collect_dict_values['author']=re.sub('\W+','-', item.split('=')[1])
        # print(re.sub('\W+','-', item.split('=')[1]))
    if 'journal' in [i.strip() for i in ii ]:
        collect_dict_values['journal']=re.sub('\W+','-', item.split('=')[1])
        
print(''.join(('article-',collect_dict_values['year'],collect_dict_values['author'],
               collect_dict_values['title'], collect_dict_values['journal'][:-1],'.pdf',)))
citation_key= [item.split(',') for item in json_string.split('\n')][0][0].split('{')[1]

print('CITATION KEY: ', citation_key)

citation_key =f'{citation_key}' # user manual input

headers = {'user-agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (HTML, like Gecko) Chrome/91.0.4472.114 Safari/537.36'}
page_contents=requests.get(url,headers=headers)
soup=BeautifulSoup(page_contents.text,'html.parser')
dsoup = copy.deepcopy(soup)
print(json_string)
print(dsoup.text[:200])

# docker run --rm --init --ulimit core=0 -p 8070:8070 lfoppiano/grobid:0.8.0


article--2005--Mohanram-Partha-S--Separating-winners-from-losers-among-lowbook-to-market-stocks-using-financial-statement-analysis--Review-of-accounting-studies.pdf
CITATION KEY:  mohanram2005separating
@article{mohanram2005separating,
  title={Separating winners from losers among lowbook-to-market stocks using financial statement analysis},
  author={Mohanram, Partha S},
  journal={Review of accounting studies},
  volume={10},
  pages={133--170},
  year={2005},
  publisher={Springer}
}
Google Scholarhtml,body,form,table,div,h1,h2,h3,h4,h5,h6,img,ol,ul,li,button{margin:0;padding:0;border:0;}table{border-collapse:collapse;border-width:0;empty-cells:show;}html,body{height:100%}#gs_top{


In [3]:
citation_dict={}
def cited_by_gscholar(dsoup):
    cited_by_link_list=[]
    for item in dsoup.find_all('div',{"class" : "gs_ri"}):
        cited_by_link_dict={}
        authors=[]
        for el in item.extract().find_all('a'):
            if 'https' in el.get('href').split(':'):
                if len(set(['View as HTML', 'Cached']) & set([str(el.text)])) ==0:
                    cited_by_link_dict['article_title']=el.text
                    cited_by_link_dict['article_link']=el.get('href')
            if '/citations?user' in el.get('href'):
                authors.append(el.text)
            elhref = el.get('href')
            if '/scholar?cites' in elhref.split('='):
                cited_by_link_dict['cited_by']=elhref
            if 'by' in el.text.split():
                #print(el.text.split())
                citation_cnt=re.findall(r'\d+', el.text)[0]
                cited_by_link_dict['citations_count']=citation_cnt
                cited_by_link_list.append(cited_by_link_dict)
        cited_by_link_dict['auhtors']=authors
    return cited_by_link_list

dsoup = copy.deepcopy(soup)
dsoup= cited_by_gscholar(dsoup)
citation_dict['citation_key']=citation_key
citation_dict['google_scholar_link']=url
citation_dict['cited_by']=dsoup
df = pd.DataFrame.from_dict([citation_dict])
df.to_csv('academic_citation_network.csv', mode='a')

!clear
!git status
!export citation_key=($citation_key)
!git add .
!git commit -m "$citation_key"
!git push

citation_dict

On branch main
Your branch is up to date with 'origin/main'.

Changes not staged for commit:
  (use "git add <file>..." to update what will be committed)
  (use "git restore <file>..." to discard changes in working directory)
	modified:   academic_citation_network.csv
	modified:   academic_citation_networks_google_scholar.ipynb

no changes added to commit (use "git add" and/or "git commit -a")


In [ ]:
import os
import xml.etree.ElementTree as ET
tree = ET.parse('/Users/krishnaneupane/Downloads/testGorbid/ssrn-3604626.grobid.tei.xml')
root = tree.getroot()
for item in root.keys():
    print(item)

{http://www.w3.org/XML/1998/namespace}space
{http://www.w3.org/2001/XMLSchema-instance}schemaLocation


In [ ]:
import xmltojson
from pprint import pprint
import json 

import xmltojson
with open('/Users/krishnaneupane/Downloads/testGorbid/ssrn-3604626.grobid.tei.xml', 'r') as f:
    my_xml = f.read()
dd= json.loads(xmltojson.parse(my_xml))

for item in dd['TEI']['text']['back']['div']:
    if 'listBibl' in list(item.keys()):

        for sitem in item['listBibl']['biblStruct']:
            journal = sitem.get('monogr').get('title').get('#text')
            try: 
                title= sitem['analytic']['title']['#text']
                for stitle in sitem['analytic']['author']:
                    fname= stitle.get('persName').get('forename')
                    if  isinstance(fname, list):
                        flname=[i['#text'] for i in fname]
                        lname= stitle['persName'].get('surname')
                        flname= flname + [lname]
                        flname= ' '.join(flname)
                        print(title,flname, journal)
                    else:
                        fname= stitle.get('persName').get('forename').get('#text')
                        lname= stitle['persName'].get('surname')
                        flname= fname +' '+ lname
                        print(title, flname, journal)
            except Exception:
                pass
            


Abnormal returns to a fundamental analysis strategy Jeffery S Abarbanell Accounting Review
Abnormal returns to a fundamental analysis strategy Brian J Bushee Accounting Review
Asset pricing with liquidity risk Viral V Acharya Journal of financial Economics
Asset pricing with liquidity risk Lasse Heje Pedersen Journal of financial Economics
Financial intermediaries and the cross-section of asset returns Tobias Adrian The Journal of Finance
Financial intermediaries and the cross-section of asset returns Erkko Etula The Journal of Finance
Financial intermediaries and the cross-section of asset returns Tyler Muir The Journal of Finance
Arbitrage risk and the bookto-market anomaly Ashiq Ali Journal of Financial Economics
Arbitrage risk and the bookto-market anomaly Lee-Seok Hwang Journal of Financial Economics
Arbitrage risk and the bookto-market anomaly Mark A Trombley Journal of Financial Economics
Asset pricing and the bid-ask spread Yakov Amihud Journal of financial Economics
Asset pric

In [ ]:
!pip install xmltojson